A. Thông tin định danh & Quản lý (Metadata)

job_id: ID duy nhất của tin tuyển dụng trên trang gốc (để tránh crawl trùng).

url: Link chi tiết của bài đăng.

crawl_at: Thời gian bạn thực hiện crawl (dùng để theo dõi độ tươi của dữ liệu).

posted_at: Ngày đăng tin hoặc ngày cập nhật của nhà tuyển dụng.

B. Nội dung công việc (Job Details)

job_title: Tiêu đề công việc (ví dụ: Senior Java Developer).

job_level: Cấp bậc (Intern, Junior, Senior, Manager...).


min: Mức lương tối thiểu.

max: Mức lương tối đa.


Lưu ý: Nên chuẩn hóa về số để dễ truy vấn $gte, $lte.

location: Thành phố (Hà Nội, TP.HCM, Remote...).

job_description: Mô tả công việc (nên lưu dạng Text để search).

job_requirements: Yêu cầu ứng viên.

skills: Mảng các kỹ năng (ví dụ: ["Python", "AWS", "ReactJS"]). Đây là trường quan trọng nhất để filter.

C. Thông tin công ty (Company Info)

company_name: Tên công ty.

company_industry: Ngành nghề.

company_size: Quy mô (ví dụ: 100-500 nhân viên).

company_address: Địa chỉ chi tiết.

In [6]:
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def get_driver():
    options = Options()
    # options.add_argument("--headless") # Chạy ẩn nếu muốn
    options.add_argument("window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    ]
    options.add_argument(f"user-agent={random.choice(user_agents)}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    # Ẩn dấu vết Selenium
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

def scroll_down(driver):
    """Cuộn trang từ từ để load hết dữ liệu/ảnh (Lazy load)"""
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    for i in range(1, total_height, 400):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(0.1)

def extract_page_data(html):
    """Xử lý HTML bằng BeautifulSoup"""
    soup = BeautifulSoup(html, 'html.parser')
    job_items = soup.select('.job-item-search-result')
    
    page_data = []
    for item in job_items:
        # Lấy ID từ attribute data-job-id
        job_id = item.get('data-job-id')
        
        # Lấy tiêu đề và Link
        title_tag = item.select_one('h3.title a')
        job_title = title_tag.get_text(strip=True) if title_tag else "N/A"
        # Xóa các tham số rác sau dấu ? trong URL
        job_url = title_tag.get('href').split('?')[0] if title_tag else "N/A"
        
        # Lấy tên công ty
        company_tag = item.select_one('.company-name, .company a')
        company_name = company_tag.get_text(strip=True) if company_tag else "N/A"
        
        # Lấy mức lương
        salary_tag = item.select_one('.title-salary, .salary')
        salary = salary_tag.get_text(strip=True) if salary_tag else "Thỏa thuận"
        
        # Lấy địa điểm (city-text)
        location_tag = item.select_one('.city-text')
        location = location_tag.get_text(strip=True) if location_tag else "N/A"

        # Lấy kinh nghiệm (exp)
        exp_tag = item.select_one('.exp span')
        experience = exp_tag.get_text(strip=True) if exp_tag else "Không yêu cầu"
        
        # Thời gian cập nhật
        time_tag = item.select_one('.label-update')
        posted_at = time_tag.get_text(strip=True).replace("Đăng ", "") if time_tag else "N/A"

        page_data.append({
            "job_id": job_id,
            "title": job_title,
            "url": job_url,
            "company": company_name,
            "salary": salary,
            "location": location,
            "exp": experience,
            "posted_at": posted_at
        })
    return page_data

def start_crawl(url):
    driver = get_driver()
    driver.get(url)
    
    all_jobs = []
    page_num = 1

    try:
        while True:
            print(f"\n--- Đang cào trang {page_num} ---")
            
            # Đợi danh sách việc làm xuất hiện
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "job-item-search-result"))
            )
            
            # Cuộn xuống để đảm bảo mọi thứ được load
            scroll_down(driver)
            time.sleep(1)
            
            # Trích xuất dữ liệu
            jobs = extract_page_data(driver.page_source)
            for j in jobs:
                print(f"[{j['job_id']}] {j['title']} - {j['company']} ({j['salary']})")
            
            all_jobs.extend(jobs)

            # Tìm nút Next
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, "ul.pagination li a[rel='next']")
                # Cuộn tới nút Next để click
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
                time.sleep(1)
                next_btn.click()
                page_num += 1
                # Nghỉ ngẫu nhiên để tránh bot detection
                time.sleep(random.uniform(2, 4))
            except:
                print("\n>>> Đã hết trang hoặc không tìm thấy nút Next.")
                break
                
    finally:
        driver.quit()
        print(f"\nTổng cộng đã cào được {len(all_jobs)} việc làm.")
        return all_jobs

# Chạy thử
if __name__ == "__main__":
    search_url = "https://www.topcv.vn/tim-viec-lam-cong-nghe-thong-tin-cr257"
    results = start_crawl(search_url)

C:\Users\Admin\AppData\Local\Temp\ipykernel_55844\1812179488.py:130: SyntaxWarning: 'return' in a 'finally' block
  return all_jobs



--- Đang cào trang 1 ---
[1988311] Nhân Viên Kinh Doanh Phần Mềm, Hoa Hồng Hấp Dẫn -Thu Nhập Lên Đến 50 Triệu - Công Ty Cổ phần Phần Mềm Hoàn Cầu (Tới 50 triệu)
[1990096] Solution Sales Specialist - Magenest.,JSC (Thoả thuận)
[1980253] Pre-Sales Kỹ Thuật (AV, ELV, Công Nghệ Thông Tin) - Cầu Giấy, Hà Nội - Lương Hấp Dẫn - CÔNG TY CỔ PHẦN ĐẦU TƯ HCOM (Thoả thuận)
[1980466] Chuyên Viên Phân Tích Nghiệp Vụ Và Điều Phối Dự Án (BA) - CÔNG TY CỔ PHẦN TẬP ĐOÀN GIỐNG CÂY TRỒNG VIỆT NAM (Thoả thuận)
[1980046] Business Analyst (BA) - Chỉ Tuyển Nam - Quận Bình Thạnh (Cũ- TPHCM) Đi Làm Ngay - Công Ty TNHH NASYS (14 - 18 triệu)
[1980091] Nhân Viên Vận Hành Máy In UV - Không Yêu Cầu Kinh Nghiệm, Quận 12 (Thu Nhập 8 - 12 Triệu + Phụ Cấp) - CÔNG TY TNHH QUÀ TẶNG HƯNG VIỆT MỸ (8 - 12 triệu)
[1980093] Backend Developer Công Ty IT Hàn Quốc - ORBRO Inc. (Thoả thuận)
[1946527] Nhân Viên Xử Lý Dữ Liệu AI Trainer – Audio (English C1 - Không Yêu Cầu Kinh Nghiệm) - CÔNG TY TNHH APPEN DATA TECHNOLOGY VIỆT NAM (

In [12]:
print(results[11])

{'job_id': '1980580', 'title': 'Data And Analytics Division', 'url': 'https://www.topcv.vn/brand/techcombank/tuyen-dung/data-and-analytics-division-j1980580.html', 'company': 'NGÂN HÀNG THƯƠNG MẠI CỔ PHẦN KỸ THƯƠNG VIỆT NAM', 'salary': 'Thoả thuận', 'location': 'Hà Nội', 'exp': 'Trên 5 năm', 'posted_at': 'Đăng1 tuần trước'}


In [13]:
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def get_driver():
    options = Options()
    # options.add_argument("--headless") 
    options.add_argument("window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    ]
    options.add_argument(f"user-agent={random.choice(user_agents)}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

def scroll_down(driver):
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    for i in range(1, total_height, 400):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(0.1)

def extract_page_data(html):
    """Xử lý HTML bằng BeautifulSoup"""
    soup = BeautifulSoup(html, 'html.parser')
    job_items = soup.select('.job-item-search-result')
    
    page_data = []
    for item in job_items:
        # Lấy ID từ attribute data-job-id
        job_id = item.get('data-job-id')
        
        # Lấy tiêu đề và Link
        title_tag = item.select_one('h3.title a')
        job_title = title_tag.get_text(strip=True) if title_tag else "N/A"
        job_url = title_tag.get('href').split('?')[0] if title_tag else "N/A"
        
        # Lấy tên công ty
        company_tag = item.select_one('.company-name, .company a')
        company_name = company_tag.get_text(strip=True) if company_tag else "N/A"
        
        # Lấy mức lương
        salary_tag = item.select_one('.title-salary, .salary')
        salary = salary_tag.get_text(strip=True) if salary_tag else "Thỏa thuận"
        
        # Lấy địa điểm (city-text)
        location_tag = item.select_one('.city-text')
        location = location_tag.get_text(strip=True) if location_tag else "N/A"

        # Lấy kinh nghiệm (exp)
        exp_tag = item.select_one('.exp span')
        experience = exp_tag.get_text(strip=True) if exp_tag else "Không yêu cầu"
        
        #  LẤY SKILLS (TAGS) ---
        # TopCV hiển thị tags trong class .tag, mỗi tag là một .item-tag
        tag_elements = item.select('.tag .item-tag')
        skills = [tag.get_text(strip=True) for tag in tag_elements]
        
        time_tag = item.select_one('.label-update')
        posted_at = time_tag.get_text(strip=True).replace("Đăng ", "") if time_tag else "N/A"

        page_data.append({
            "job_id": job_id,
            "title": job_title,
            "url": job_url,
            "company": company_name,
            "salary": salary,
            "location": location,
            "exp": experience,
            "skills": skills,  
            "posted_at": posted_at
        })
    return page_data

def start_crawl(url):
    driver = get_driver()
    driver.get(url)
    
    all_jobs = []
    page_num = 1

    try:
        while True:
            print(f"\n--- Đang cào trang {page_num} ---")
            
            # Đợi danh sách việc làm xuất hiện
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "job-item-search-result"))
            )
            # Cuộn xuống để đảm bảo mọi thứ được load
            scroll_down(driver)
            time.sleep(1)
            # Trích xuất dữ liệu
            jobs = extract_page_data(driver.page_source)
            for j in jobs:
                print(f"[{j['job_id']}] {j['title']} | Skills: {', '.join(j['skills'][:3])}...")
            
            all_jobs.extend(jobs)
            # Tìm nút Next
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, "ul.pagination li a[rel='next']")
                # Cuộn tới nút Next để click
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
                time.sleep(1)
                next_btn.click()
                page_num += 1
                # Nghỉ ngẫu nhiên để tránh bot detection
                time.sleep(random.uniform(2, 4))
            except:
                print("\n>>> Đã hết trang hoặc không tìm thấy nút Next.")
                break
                
    finally:
        driver.quit()
        print(f"\nTổng cộng đã cào được {len(all_jobs)} việc làm.")
        return all_jobs

if __name__ == "__main__":
    search_url = "https://www.topcv.vn/tim-viec-lam-cong-nghe-thong-tin-cr257"
    results = start_crawl(search_url)

<>:129: SyntaxWarning: 'return' in a 'finally' block
C:\Users\Admin\AppData\Local\Temp\ipykernel_55844\1010011133.py:129: SyntaxWarning: 'return' in a 'finally' block
  return all_jobs
C:\Users\Admin\AppData\Local\Temp\ipykernel_55844\1010011133.py:129: SyntaxWarning: 'return' in a 'finally' block
  return all_jobs



--- Đang cào trang 1 ---
[1971328] Nhân Viên Kinh Doanh/Sale/Tư Vấn Bán Hàng(Máy In)Lương Cứng 12tr+Hoa Hồng-Nhận Việc Ngay | Skills: Bán hàng kỹ thuật IT, Direct Sales, Telesale......
[1990948] Trưởng Ban Kỹ Thuật [Hà Nội] - Thu Nhập 50-100 Triệu, Nhiều Đãi Ngộ Hấp Dẫn | Skills: Chief Technology Officer (CTO), Viễn thông...
[1854296] Nhân Viên IT Phần Cứng - KInh Nghiệm Từ 2 Năm | Skills: IT Helpdesk/IT support, IT - Phần cứng và ......
[1981043] UX/UI Design - Kinh Nghiệm 1 Năm Trở Lên Tại Quận 2 | Skills: UI/UX Design, 1 năm kinh nghiệm, Cao Đẳng tr......
[1980580] Data And Analytics Division | Skills: Data Analyst, Trên 5 năm kinh nghiệm, Đại Họ......
[1978491] Lập Trình Viên Web (C#, .Net) - Từ 1 Năm Kinh Nghiệm | Skills: Fullstack Developer, IT - Phần mềm, Nghỉ thứ......
[1689299] Chuyên Viên Kiểm Thử | Skills: Chuyên môn khác, IT - Phần cứng và máy tín......
[1988101] DevOps Engineer (Junior/Mid) - Salary Up To 35,000,000 VND - Thu Duc | Skills: Software Engineer, IT - Phần mềm

In [14]:
results[0]

{'job_id': '1971328',
 'title': 'Nhân Viên Kinh Doanh/Sale/Tư Vấn Bán Hàng(Máy In)Lương Cứng 12tr+Hoa Hồng-Nhận Việc Ngay',
 'url': 'https://www.topcv.vn/viec-lam/nhan-vien-kinh-doanh-sale-tu-van-ban-hangmay-inluong-cung-12tr-hoa-hong-nhan-viec-ngay/1971328.html',
 'company': 'CÔNG TY CP THIẾT BỊ CÔNG NGHIỆP HUỲNH LONG',
 'salary': 'Thoả thuận',
 'location': 'Hồ Chí Minh (mới)',
 'exp': 'Dưới 1 năm',
 'skills': ['Bán hàng kỹ thuật IT', 'Direct Sales', 'Telesale...'],
 'posted_at': 'Đăng1 ngày trước'}

In [18]:
import re
import time
import random
from datetime import datetime, timedelta
def convert_to_million(value_str, currency_type):
    try:
        value = float(value_str)
        if currency_type == "usd":
            return round((value * 25400) / 1000000, 2)
        return round(value, 2)
    except:
        return float('nan')

def process_job_data(raw_jobs):
    processed_list = []
    today = datetime.now()
    benefit_keywords = ['thưởng', 'nghỉ', 'bảo hiểm', 'du lịch', 'phụ cấp', 'trưa', 'máy tính', 'lương tháng 13', 'teambuilding', 'chăm sóc']

    for raw in raw_jobs:
        # Xử lý thời gian
        relative_text = raw['posted_at'].lower()
        num_in_time = re.search(r'\d+', relative_text)
        number = int(num_in_time.group()) if num_in_time else 0
        if any(unit in relative_text for unit in ["giờ", "phút", "vừa xong"]):
            exact_date = today
        elif "ngày" in relative_text:
            exact_date = today - timedelta(days=number)
        elif "tuần" in relative_text:
            exact_date = today - timedelta(weeks=number)
        elif "tháng" in relative_text:
            exact_date = today - timedelta(days=number * 30)
        else:
            exact_date = today

        # Xử lý lương
        salary_raw = raw['salary'].upper()
        min_salary = float('nan')
        max_salary = float('nan')
        currency_type = "usd" if "USD" in salary_raw else "vnd"
        clean_salary_text = salary_raw.replace('.', '').replace(',', '')
        numbers = re.findall(r'\d+', clean_salary_text)

        if "THỎA THUẬN" in salary_raw or not numbers:
            pass 
        elif "TỪ" in salary_raw and len(numbers) == 1:
            min_salary = convert_to_million(numbers[0], currency_type)
        elif any(x in salary_raw for x in ["TỚI", "ĐẾN", "LÊN ĐẾN"]) and len(numbers) == 1:
            max_salary = convert_to_million(numbers[0], currency_type)
        elif len(numbers) >= 2:
            min_salary = convert_to_million(numbers[0], currency_type)
            max_salary = convert_to_million(numbers[1], currency_type)
        elif len(numbers) == 1:
            min_salary = convert_to_million(numbers[0], currency_type)

        # Phân loại tags
        skills = []
        benefits = []
        for tag in raw['skills']:
            if any(kw in tag.lower() for kw in benefit_keywords):
                benefits.append(tag)
            else:
                skills.append(tag)

        processed_list.append({
            "job_id": raw['job_id'],
            "title": raw['title'],
            "company": raw['company'],
            "url": raw['url'],
            "salary_raw": raw['salary'],
            "min_salary_mil": min_salary,
            "max_salary_mil": max_salary,
            "skills": skills,
            "benefits": benefits,
            "posted_date": exact_date.strftime('%Y-%m-%d'),
            "processed_at": today.strftime('%Y-%m-%d %H:%M:%S')
        })
    return processed_list

if __name__ == "__main__":
    processed_data = process_job_data(results)
    
    # 3. Hiển thị kết quả
    print("\nKẾT QUẢ SAU XỬ LÝ:")
    for job in processed_data:
        print(f"--- {job['title']} ---")
        print(f"Công ty: {job['company']}")
        print(f"Lương Min: {job['min_salary_mil']}tr | Max: {job['max_salary_mil']}tr")
        print(f"Kỹ năng: {job['skills']}")
        print(f"Ngày đăng: {job['posted_date']}")


KẾT QUẢ SAU XỬ LÝ:
--- Nhân Viên Kinh Doanh/Sale/Tư Vấn Bán Hàng(Máy In)Lương Cứng 12tr+Hoa Hồng-Nhận Việc Ngay ---
Công ty: CÔNG TY CP THIẾT BỊ CÔNG NGHIỆP HUỲNH LONG
Lương Min: nantr | Max: nantr
Kỹ năng: ['Bán hàng kỹ thuật IT', 'Direct Sales', 'Telesale...']
Ngày đăng: 2025-12-22
--- Trưởng Ban Kỹ Thuật [Hà Nội] - Thu Nhập 50-100 Triệu, Nhiều Đãi Ngộ Hấp Dẫn ---
Công ty: CÔNG TY CỔ PHẦN BKSCITECH
Lương Min: 50.0tr | Max: 100.0tr
Kỹ năng: ['Chief Technology Officer (CTO)', 'Viễn thông']
Ngày đăng: 2025-12-20
--- Nhân Viên IT Phần Cứng - KInh Nghiệm Từ 2 Năm ---
Công ty: CÔNG TY TNHH MỘT THÀNH VIÊN THÉP VAS AN HƯNG TƯỜNG
Lương Min: 12.0tr | Max: 13.0tr
Kỹ năng: ['IT Helpdesk/IT support', 'IT - Phần cứng và ...']
Ngày đăng: 2025-12-16
--- UX/UI Design - Kinh Nghiệm 1 Năm Trở Lên Tại Quận 2 ---
Công ty: CÔNG TY CỔ PHẦN TMDV THỦY SẢN MIỀN NAM
Lương Min: nantr | Max: nantr
Kỹ năng: ['UI/UX Design', '1 năm kinh nghiệm', 'Cao Đẳng tr...']
Ngày đăng: 2025-12-16
--- Data And Analytics Divis

In [20]:
processed_data[1]

{'job_id': '1990948',
 'title': 'Trưởng Ban Kỹ Thuật [Hà Nội] - Thu Nhập 50-100 Triệu, Nhiều Đãi Ngộ Hấp Dẫn',
 'company': 'CÔNG TY CỔ PHẦN BKSCITECH',
 'url': 'https://www.topcv.vn/viec-lam/truong-ban-ky-thuat-ha-noi-thu-nhap-50-100-trieu-nhieu-dai-ngo-hap-dan/1990948.html',
 'salary_raw': '50 - 100 triệu',
 'min_salary_mil': 50.0,
 'max_salary_mil': 100.0,
 'skills': ['Chief Technology Officer (CTO)', 'Viễn thông'],
 'benefits': [],
 'posted_date': '2025-12-20',
 'processed_at': '2025-12-23 11:40:40'}